# Lab 6


    Part 1: Cross-validation of ridge regression

In [18]:
import numpy as np
from math import sqrt


#set up folds and groups
def foldgen():

    train = []
    
    data = np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1)

    #split data into folds
    foldlen = int(len(data) / 5)
    test = []
    for i in range(5):
        test.append(data[ (foldlen * i) : (foldlen * (i + 1)) ])
        
    #set up groups
    for i in range(5):
        included = list(range(5))
        del included[i]
        
        group = np.zeros((0,96),np.float64)
        for j in included:
            group = np.concatenate((group,test[j]))
        train.append(group)
    
    return((train,test))
        
    
#ridge regression fit test: returns rmse given training data, testing data, and lambda
def ridgeRegRmse(train, test, lam):
    
    #split into features and result
    trainsplit = np.split(train, [1], 1)
    b = np.full((len(trainsplit[0]), 1), 1)
    trainX = np.append(trainsplit[1], b, axis = 1)
    trainY = trainsplit[0]
    
    testsplit = np.split(test, [1], 1)
    b1 = np.full((len(testsplit[0]), 1), 1)
    testX = np.append(testsplit[1], b1, axis = 1)
    testY = testsplit[0]
    
    
    #calculate weights (broken down for readability)
    diag = np.zeros((len(trainX[0]), len(trainX[0])), dtype = np.float64)
    for i in range(len(trainX[0])):
        diag[i][i] = lam
    
    trainXt = np.transpose(trainX)
    xtxprod = np.matmul(trainXt, trainX) + diag
    invm = np.linalg.inv(xtxprod)
    invmxt = np.matmul(invm, trainXt)
    w = np.matmul(invmxt, trainY)
    
    
    #generate predictions
    result = np.matmul(testX, w)
    
    
    #calculate rmse
    rmse = 0
    for i in range(len(testX)):
        rmse += (result[i][0] - testY[i][0]) ** 2
    rmse /= len(testX)
    
    rmse = sqrt(rmse)
    
    return rmse


    
result = foldgen()
train = result[0]
test = result[1]

lam = 400

for i in range(10):
    
    rmse = 0
    
    for i in range(5):
        rmse += ridgeRegRmse(train[i],test[i],lam)
    
    rmse /= 5
    
    print("For lambda = " + str(lam) + ", RMSE = " + str(rmse))
    
    lam /= 2
    

trainrmse = ridgeRegRmse( (np.concatenate((train[0], test[0]))) , np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1) , 25)
print("\nUsing lambda 25, rmse on train data is " + str(trainrmse))

testrmse = ridgeRegRmse( (np.concatenate((train[0], test[0]))) , np.loadtxt(fname = 'crime-test.txt', dtype = np.float64, delimiter = '\t', skiprows = 1) , 25)
print("\nUsing lambda 25, rmse on test data is " + str(testrmse))


For lambda = 400, RMSE = 0.14951277619075795
For lambda = 200.0, RMSE = 0.14069401845317783
For lambda = 100.0, RMSE = 0.13727677223186574
For lambda = 50.0, RMSE = 0.13615594392755503
For lambda = 25.0, RMSE = 0.13591585919851962
For lambda = 12.5, RMSE = 0.13602237446230653
For lambda = 6.25, RMSE = 0.13626786494076776
For lambda = 3.125, RMSE = 0.1365709569364274
For lambda = 1.5625, RMSE = 0.13688764073197515
For lambda = 0.78125, RMSE = 0.13717626806866784

Using lambda 25, rmse on train data is 0.1287970145987978

Using lambda 25, rmse on test data is 0.14574650707058048


    Part 2: Linear regression using gradient descent

In [21]:
#returns gradient descent weights given alpha, sample features, and sample results
def getLRWeight(alpha, x, y):
    
    #initializing initial weights randomly from a gaussian distribution
    weight = np.zeros((96,1), np.float64)

    for i in range(96):
        weight[i][0] = np.random.normal(0, 1)
        
    
    #first iteration
    weight1 = weight + alpha * np.matmul( np.transpose(x) , (y - np.matmul( x , weight)))
    
    #further iterations
    while ((np.linalg.norm(weight) - np.linalg.norm(weight1)) > (10 ** -5)):
        weight = weight1
        weight1 = weight + alpha * np.matmul( np.transpose(x) , (y - np.matmul( x , weight)))
        
    return weight1



#calculate rmse for a given set of weights, features, and values
def getRMSE(w, testX, testY):

    #generate predictions
    result = np.matmul(testX, w)
    
    
    #calculate rmse
    rmse = 0
    for i in range(len(testX)):
        rmse += (result[i][0] - testY[i][0]) ** 2
    rmse /= len(testX)
    rmse = sqrt(rmse)
    
    return rmse


#find predictions for a given sample
def problem2(samples):
    
    train = np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1)
    trainsplit = np.split(train, [1], 1)
    b = np.full((len(trainsplit[0]), 1), 1)
    trainX = np.append(trainsplit[1], b, axis = 1)
    trainY = trainsplit[0]
    
    weights = getLRWeight((10 ** -5), trainX, trainY)
    
    #generate predictions
    return np.matmul(samples, weights)





train = np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1)
trainsplit = np.split(train, [1], 1)
b = np.full((len(trainsplit[0]), 1), 1)
trainX = np.append(trainsplit[1], b, axis = 1)
trainY = trainsplit[0]

test = np.loadtxt(fname = 'crime-test.txt', dtype = np.float64, delimiter = '\t', skiprows = 1)
testsplit = np.split(test, [1], 1)
b1 = np.full((len(testsplit[0]), 1), 1)
testX = np.append(testsplit[1], b1, axis = 1)
testY = testsplit[0]

weights = getLRWeight((10 ** -5), trainX, trainY)
        
print("Training RMSE: " + str(getRMSE(weights, trainX, trainY)))
print("Testing RMSE: " + str(getRMSE(weights, testX, testY)))

#print(problem2(testX))

Training RMSE: 0.12960691391283416
Testing RMSE: 0.146224653669273


    Part 3: Cross-validated ridge regression using gradient descent

In [23]:
#set up folds and groups
def foldgen():

    train = []
    
    data = np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1)

    #split data into folds
    foldlen = int(len(data) / 5)
    test = []
    for i in range(5):
        test.append(data[ (foldlen * i) : (foldlen * (i + 1)) ])
        
    #set up groups
    for i in range(5):
        included = list(range(5))
        del included[i]
        
        group = np.zeros((0,96),np.float64)
        for j in included:
            group = np.concatenate((group,test[j]))
        train.append(group)
    
    return((train,test))



#returns gradient descent weights given alpha, sample features, sample results, and lambda
def getRRWeight(alpha, x, y, lam):
    
    #initializing initial weights randomly from a gaussian distribution
    weight = np.zeros((96,1), np.float64)

    for i in range(96):
        weight[i][0] = np.random.normal(0, 1)
        
        
    
    #first iteration
    weight1 = weight + alpha * (np.matmul( np.transpose(x) , (y - np.matmul( x , weight)) ) - (lam * weight))
    
    #further iterations
    while ((np.linalg.norm(weight - weight1)) > (10 ** -5)):
        weight = weight1
        weight1 = weight + alpha * np.matmul( np.transpose(x) , (y - np.matmul( x , weight)))
        
    return weight1
        
    
#ridge regression fit test: returns rmse given training data, testing data, and lambda
def ridgeRegRmse(train, test, lam):
    
    #split into features and result
    trainsplit = np.split(train, [1], 1)
    b = np.full((len(trainsplit[0]), 1), 1)
    trainX = np.append(trainsplit[1], b, axis = 1)
    trainY = trainsplit[0]
    
    testsplit = np.split(test, [1], 1)
    b1 = np.full((len(testsplit[0]), 1), 1)
    testX = np.append(testsplit[1], b1, axis = 1)
    testY = testsplit[0]
    
    #calculate weights
    w = getRRWeight((10 ** -5) , trainX, trainY, lam)
    
    
    #generate predictions
    result = np.matmul(testX, w)
    
    
    #calculate rmse
    rmse = 0
    for i in range(len(testX)):
        rmse += (result[i][0] - testY[i][0]) ** 2
    rmse /= len(testX)
    
    rmse = sqrt(rmse)
    
    return rmse


    
result = foldgen()
train = result[0]
test = result[1]

lam = 800

for i in range(14):
    
    rmse = 0
    
    for i in range(5):
        rmse += ridgeRegRmse(train[i],test[i],lam)
    
    rmse /= 5
    
    print("For lambda = " + str(lam) + ", RMSE = " + str(rmse))
    
    lam /= 2
    

trainrmse = ridgeRegRmse( (np.concatenate((train[0], test[0]))) , np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1) , 0.78125)
print("\nUsing lambda 0.78125, rmse on train data is " + str(trainrmse))

testrmse = ridgeRegRmse( (np.concatenate((train[0], test[0]))) , np.loadtxt(fname = 'crime-test.txt', dtype = np.float64, delimiter = '\t', skiprows = 1) , 0.78125)
print("\nUsing lambda 0.78125, rmse on test data is " + str(testrmse))

For lambda = 800, RMSE = 0.1394825846291256
For lambda = 400.0, RMSE = 0.1401574499641466
For lambda = 200.0, RMSE = 0.1415926993500193
For lambda = 100.0, RMSE = 0.13948865847596398
For lambda = 50.0, RMSE = 0.1410199725173026
For lambda = 25.0, RMSE = 0.1409041336861009
For lambda = 12.5, RMSE = 0.13920392488009062
For lambda = 6.25, RMSE = 0.13971730524619524
For lambda = 3.125, RMSE = 0.13907924732676508
For lambda = 1.5625, RMSE = 0.13918784673092294
For lambda = 0.78125, RMSE = 0.13959912486900922
For lambda = 0.390625, RMSE = 0.14178158030626964
For lambda = 0.1953125, RMSE = 0.13951262841357573
For lambda = 0.09765625, RMSE = 0.1381478680388374

Using lambda 0.78125, rmse on train data is 0.12907069317876615

Using lambda 0.78125, rmse on test data is 0.1464145946505155


In [11]:
def problem3(samples):
    
    train = np.loadtxt(fname = 'crime-train.txt', dtype = np.float64, delimiter = '\t', skiprows = 1)
    trainsplit = np.split(train, [1], 1)
    b = np.full((len(trainsplit[0]), 1), 1)
    trainX = np.append(trainsplit[1], b, axis = 1)
    trainY = trainsplit[0]
    
    weights = getRRWeight((10 ** -5), trainX, trainY, 400)
    
    #generate predictions
    return np.matmul(samples, weights)




print(problem3(testX))

[[ 5.59532331e-02]
 [ 2.44708449e-01]
 [ 1.03237057e-01]
 [ 1.38456571e-01]
 [ 1.30840553e-01]
 [ 1.23808893e-01]
 [ 7.41651593e-01]
 [ 2.29813113e-01]
 [ 1.98280026e-01]
 [ 5.20235350e-01]
 [ 6.89200394e-02]
 [ 1.34248943e-01]
 [ 4.30522202e-01]
 [ 6.72167214e-02]
 [ 1.66135840e-01]
 [ 3.19661621e-01]
 [ 2.77917902e-01]
 [-1.87740094e-02]
 [ 6.17443759e-01]
 [-3.93325813e-02]
 [ 1.86765630e-01]
 [ 1.78578347e-01]
 [ 6.17725455e-03]
 [ 1.39556626e-01]
 [ 3.99386768e-02]
 [-2.47291783e-02]
 [ 5.82716951e-02]
 [ 4.44310039e-01]
 [ 7.52358645e-03]
 [ 3.40418346e-01]
 [ 2.69036193e-01]
 [ 1.21321465e-01]
 [ 1.38105080e-01]
 [ 1.39194517e-01]
 [ 2.47763352e-01]
 [ 4.26352723e-01]
 [ 1.08546339e+00]
 [ 4.38928262e-01]
 [ 2.34255929e-01]
 [ 6.94633386e-02]
 [ 9.81857392e-02]
 [ 1.79058865e-01]
 [ 2.93356864e-01]
 [ 7.74841220e-01]
 [ 1.85164869e-01]
 [ 3.87351525e-01]
 [ 6.70302026e-01]
 [ 2.97555952e-02]
 [-7.97578133e-02]
 [ 2.10992298e-01]
 [ 2.09297459e-01]
 [ 3.83565240e-01]
 [ 2.4735306